In [2]:
%matplotlib inline
import librosa
import IPython.display as ipd
import pandas as pd
import os
import warnings
from importlib import reload
import sys
sys.path.append('..')
warnings.filterwarnings('ignore')

from lid import demoutils

In [3]:
store_fld = "/Users/a18180846/projects/data"
N = 30
df = pd.read_csv('../data/preprocessed_valid.csv')
df = df.groupby('label', as_index=False).apply(lambda x: x.sample(N)).reset_index(drop=True)
df['npy'] = df.apply(lambda row: os.path.join(store_fld, row.label, 'preprocess', row.path), axis=1)
df['mp3'] = df.apply(lambda row: os.path.join(store_fld, row.label, 'clips', row.path.replace('npy', 'mp3')), axis=1)

print(df.label.value_counts())
df.head()

ru    30
en    30
de    30
Name: label, dtype: int64


,path,label,duration_ms,npy,mp3
0,e4ea07520f374aaae5b8a672a111430de68b9575f26a2b...,de,3600,/Users/a18180846/projects/data/de/preprocess/e...,/Users/a18180846/projects/data/de/clips/e4ea07...
1,9915ff6fdf6c22ce250b7d303bef741acc909046636a54...,de,3270,/Users/a18180846/projects/data/de/preprocess/9...,/Users/a18180846/projects/data/de/clips/9915ff...
2,b208a71ff5fb47ab192a071845f2a2ea06f0c0d335c5b3...,de,4860,/Users/a18180846/projects/data/de/preprocess/b...,/Users/a18180846/projects/data/de/clips/b208a7...
3,b650f9c499cdfab068ea4335a6ccd1f24a62c71bda5f52...,de,5730,/Users/a18180846/projects/data/de/preprocess/b...,/Users/a18180846/projects/data/de/clips/b650f9...
4,69284c18ec8f7fc1ecda5e8e856408dd3c94056531eaa1...,de,2730,/Users/a18180846/projects/data/de/preprocess/6...,/Users/a18180846/projects/data/de/clips/69284c...


In [18]:
reload(demoutils)

user_settings = {'vad_level': 1, 'noise_reduction': True,
                 'voted_seconds': 10, 'voting_overlap': 0.5, 'voting': 'mean',
                'store': "/Users/a18180846/projects/data"}
setting_path = r"../experiments/ldcnn32mel512hop.yml"
predictor = demoutils.LID(setting_path, user_settings)

def test(lang, index):
    audio_path = df[df.label == lang].iloc[index].mp3

    x, sr = librosa.load(audio_path)
    (label, p), window_labels = predictor.predict(x, sr, False)

    print(audio_path)
    print(f"{len(x) / sr:.2f}s, sr={sr}", end='\n\n')
    print(f"Predict: '{label}'  probability={p:.2f}\n\tframes: {window_labels}")
    return ipd.Audio(data=x, rate=sr)

In [19]:
test('ru', 1)

/Users/a18180846/projects/data/ru/clips/common_voice_ru_20149260.mp3
6.65s, sr=22050

Predict: 'ru'  probability=0.53
	frames: ['en', 'ru', 'ru', 'ru', 'ru', 'en', 'ru', 'ru', 'de', 'ru', 'ru', 'en', 'de']


In [28]:
test('ru', 4)

/Users/a18180846/projects/data/ru/clips/common_voice_ru_18888355.mp3
6.70s, sr=22050

Predict: 'ru'  probability=0.48
	frames: ['ru', 'en', 'en', 'ru', 'ru', 'ru', 'ru', 'ru', 'en', 'de', 'ru', 'en', 'de']


In [27]:
test('en', 1)

/Users/a18180846/projects/data/en/clips/0be7439cef629dfac781f74ff18c5a47c79edc84c366ab517919a34969e034d0094e88d64524c0f74dfb270f96cf6716e82018c083c7823b1f7eb618818f925c.mp3
4.22s, sr=22050

Predict: 'en'  probability=0.42
	frames: ['en', 'en', 'ru', 'ru', 'en', 'en', 'en', 'de']


In [25]:
test('en', 4)

/Users/a18180846/projects/data/en/clips/0b165057f1f7f65fcbfe348c291dca998169b42193ab3815b59d1f3a3f59dd326a730dfe6d1658385e064b36a91b23ea9a079390490a8bd24b9f7f47e3acd956.mp3
2.74s, sr=22050

Predict: 'en'  probability=0.51
	frames: ['en', 'en', 'en', 'de', 'en']


In [21]:
test('de', 1)

/Users/a18180846/projects/data/de/clips/9915ff6fdf6c22ce250b7d303bef741acc909046636a54e02ce7cc9505dbe54b2bb16630719ba706f6f2953e79f42c9f72f8fa773f68dfe82aca94fc68f274f1.mp3
4.13s, sr=22050

Predict: 'de'  probability=0.42
	frames: ['ru', 'ru', 'de', 'de', 'de', 'ru']


In [29]:
test('de', 4)

/Users/a18180846/projects/data/de/clips/69284c18ec8f7fc1ecda5e8e856408dd3c94056531eaa118961c839338ecd85483f76cb7220c1e807df9f90a06cd1befe10f59924ea43784bb5a1061243838e0.mp3
3.10s, sr=22050

Predict: 'de'  probability=0.39
	frames: ['en', 'de', 'de', 'ru', 'ru', 'en']
